In [2]:
### imports

# external modules
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import DataLoader
importlib.reload(DataLoader)
import HistStruct
importlib.reload(HistStruct)

2022-06-16 23:22:31.837181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.

<module 'HistStruct' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/HistStruct.py'>

In [3]:
### Set Data Controls

# Select all available histograms automatically or user selection
automatic_histnames = True

year = '2017' # Data-taking year
eras = ['B', 'C', 'D', 'E', 'F'] # List of eras
dim = 1 # Dimension of histograms (1D or 2D)
# May need alteration later, but this is the only data directory I have perms for
datadir = '/eos/project/c/cmsml4dc/ML_2020/UL2017_Data/'
outputdir = '../data/'

histnames = []
# Case user wants to select histnames manually
if not automatic_histnames:
    # Manual selector for histograms
    histnames = ([
        'NormalizedHitResiduals_TIB__Layer__1',
        'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1',
        'NormalizedHitResiduals_TIB__Layer__2',
        'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2',
        'NormalizedHitResiduals_TIB__Layer__3',
        'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3',
        'NormalizedHitResiduals_TIB__Layer__4',
        'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4',
        'NormalizedHitResiduals_TIB__Layer__5',
        'Summary_ClusterStoNCorr__OnTrack__TIB__layer__5',
        'NormalizedHitResiduals_TIB__Layer__6',
        'Summary_ClusterStoNCorr__OnTrack__TIB__layer__6',
        'chargeInner_PXLayer_1',
        'chargeInner_PXLayer_2',
        'chargeInner_PXLayer_3',
        'chargeInner_PXLayer_4',
        'chargeOuter_PXLayer_1',
        'chargeOuter_PXLayer_2',
        'chargeOuter_PXLayer_3',
        'chargeOuter_PXLayer_4',
        
    ])

    print('Histograms to process: ' + str(len(histnames)))

In [4]:
### Read and Format CSV Files
# CSV files are formatted into CSvs with one histogram type each (with the full year's data)
# Note: This cell can take quite a while to run!

# Loop over eras
for era in eras:
    print('\nBegin processing for era ' + era)
    
    # Create a DataLoader instance
    dloader = DataLoader.DataLoader()

    # Get the CSV files (if the line above is altered to be a list, remove the brackets)
    csvfiles = dloader.get_csv_files_in_dirs([datadir + 'DF' + year + era + '_1D_Complete/'])

    # Automatic histogram selection
    if automatic_histnames:
        df = dloader.get_dataframe_from_files(csvfiles)
        histnames_with_duplicates = df['hname'].values
        
        # Removes duplicate histograms
        for histname in histnames_with_duplicates:
            if not (histname in histnames):
                histnames.append(histname)
        del(histnames_with_duplicates)
              
    else:
        df = dloader.get_dataframe_from_files(csvfiles, histnames = histnames)
                
    # Loop over histnames and write one file per histogram type
    iterate = 1
    for histname in histnames:
        thisdf = dfu.select_histnames(df, [histname])
        
        # Save the histogram for use
        outputfile = 'DF' + year + era + '_' + histname + '.csv'
        print('\nCreating file ' + outputdir + year + era + '/'+ outputfile + ' - File ' + str(iterate) + '/' + str(len(histnames)))
        dloader.write_dataframe_to_file(thisdf, os.path.join((outputdir + year + era + '/'), outputfile))
        iterate = iterate + 1
    del(dloader)


Begin processing for era B
INFO in csv_utils.py / read_and_merge_csv: reading and merging 33 csv files...
  - now processing file 1 of 33...
  - now processing file 2 of 33...
  - now processing file 3 of 33...
  - now processing file 4 of 33...
  - now processing file 5 of 33...
  - now processing file 6 of 33...
  - now processing file 7 of 33...
  - now processing file 8 of 33...
  - now processing file 9 of 33...
  - now processing file 10 of 33...
  - now processing file 11 of 33...
  - now processing file 12 of 33...
  - now processing file 13 of 33...
  - now processing file 14 of 33...
  - now processing file 15 of 33...
  - now processing file 16 of 33...
  - now processing file 17 of 33...
  - now processing file 18 of 33...
  - now processing file 19 of 33...
  - now processing file 20 of 33...
  - now processing file 21 of 33...
  - now processing file 22 of 33...
  - now processing file 23 of 33...
  - now processing file 24 of 33...
  - now processing file 25 of 33...
  


Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2.csv - File 79/133

Creating file ../data/2017B/DF2017B_NormalizedHitResiduals_TID__wheel__3.csv - File 80/133

Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3.csv - File 81/133

Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1.csv - File 82/133

Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2.csv - File 83/133

Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3.csv - File 84/133

Creating file ../data/2017B/DF2017B_NormalizedHitResiduals_TOB__Layer__1.csv - File 85/133

Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TOB__layer__1.csv - File 86/133

Creating file ../data/2017B/DF2017B_NormalizedHitResiduals_TOB__Layer__2.csv - File 87/133

Creating file ../data/2017B/DF2017B_Summary_ClusterStoNCorr__OnTrack__TOB

  - now processing file 72 of 72...
INFO in csv_utils.py / read_and_merge_csv: merged 72 csv files.

Creating file ../data/2017C/DF2017C_goodvtxNbr.csv - File 1/133
WARNING in DataLoader.write_dataframe_to_file: the output directory ../data/2017C does not exist yet; will try to create it.

Creating file ../data/2017C/DF2017C_adc_PXLayer_1.csv - File 2/133

Creating file ../data/2017C/DF2017C_adc_PXLayer_2.csv - File 3/133

Creating file ../data/2017C/DF2017C_adc_PXLayer_3.csv - File 4/133

Creating file ../data/2017C/DF2017C_adc_PXLayer_4.csv - File 5/133

Creating file ../data/2017C/DF2017C_adc_PXDisk_+1.csv - File 6/133

Creating file ../data/2017C/DF2017C_adc_PXDisk_+2.csv - File 7/133

Creating file ../data/2017C/DF2017C_adc_PXDisk_+3.csv - File 8/133

Creating file ../data/2017C/DF2017C_adc_PXDisk_-1.csv - File 9/133

Creating file ../data/2017C/DF2017C_adc_PXDisk_-2.csv - File 10/133

Creating file ../data/2017C/DF2017C_adc_PXDisk_-3.csv - File 11/133

Creating file ../data/2017C


Creating file ../data/2017C/DF2017C_NormalizedHitResiduals_TOB__Layer__4.csv - File 91/133

Creating file ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TOB__layer__4.csv - File 92/133

Creating file ../data/2017C/DF2017C_NormalizedHitResiduals_TOB__Layer__5.csv - File 93/133

Creating file ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TOB__layer__5.csv - File 94/133

Creating file ../data/2017C/DF2017C_NormalizedHitResiduals_TOB__Layer__6.csv - File 95/133

Creating file ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TOB__layer__6.csv - File 96/133

Creating file ../data/2017C/DF2017C_Chi2oNDF_lumiFlag_GenTk.csv - File 97/133

Creating file ../data/2017C/DF2017C_NumberOfRecHitsPerTrack_lumiFlag_GenTk.csv - File 98/133

Creating file ../data/2017C/DF2017C_NumberOfTracks_lumiFlag_GenTk.csv - File 99/133

Creating file ../data/2017C/DF2017C_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__1.csv - File 100/133

Creating file ../data/2017C/DF2017C_Summary_TotalNu


Creating file ../data/2017D/DF2017D_size_PXDisk_+3.csv - File 34/133

Creating file ../data/2017D/DF2017D_size_PXDisk_-1.csv - File 35/133

Creating file ../data/2017D/DF2017D_size_PXDisk_-2.csv - File 36/133

Creating file ../data/2017D/DF2017D_size_PXDisk_-3.csv - File 37/133

Creating file ../data/2017D/DF2017D_MainDiagonal Position.csv - File 38/133

Creating file ../data/2017D/DF2017D_NumberOfClustersInPixel.csv - File 39/133

Creating file ../data/2017D/DF2017D_NumberOfClustersInStrip.csv - File 40/133

Creating file ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__wheel__1.csv - File 41/133

Creating file ../data/2017D/DF2017D_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1.csv - File 42/133

Creating file ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__wheel__2.csv - File 43/133

Creating file ../data/2017D/DF2017D_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2.csv - File 44/133

Creating file ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__wheel__3.csv - F


Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TIB__layer__1.csv - File 118/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TIB__layer__2.csv - File 119/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TIB__layer__3.csv - File 120/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TIB__layer__4.csv - File 121/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TID__MINUS__wheel__1.csv - File 122/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TID__MINUS__wheel__2.csv - File 123/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TID__MINUS__wheel__3.csv - File 124/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TID__PLUS__wheel__1.csv - File 125/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TID__PLUS__wheel__2.csv - File 126/133

Creating file ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TID__PLUS__wheel__3.


Creating file ../data/2017E/DF2017E_NormalizedHitResiduals_TEC__wheel__9.csv - File 57/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9.csv - File 58/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1.csv - File 59/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2.csv - File 60/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__3.csv - File 61/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__4.csv - File 62/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__5.csv - File 63/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__6.csv - File 64/133

Creating file ../data/2017E/DF2017E_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__7.csv - File 65/133

Creating file ../data/2017E/DF2017

  - now processing file 9 of 73...
  - now processing file 10 of 73...
  - now processing file 11 of 73...
  - now processing file 12 of 73...
  - now processing file 13 of 73...
  - now processing file 14 of 73...
  - now processing file 15 of 73...
  - now processing file 16 of 73...
  - now processing file 17 of 73...
  - now processing file 18 of 73...
  - now processing file 19 of 73...
  - now processing file 20 of 73...
  - now processing file 21 of 73...
  - now processing file 22 of 73...
  - now processing file 23 of 73...
  - now processing file 24 of 73...
  - now processing file 25 of 73...
  - now processing file 26 of 73...
  - now processing file 27 of 73...
  - now processing file 28 of 73...
  - now processing file 29 of 73...
  - now processing file 30 of 73...
  - now processing file 31 of 73...
  - now processing file 32 of 73...
  - now processing file 33 of 73...
  - now processing file 34 of 73...
  - now processing file 35 of 73...
  - now processing file 36 of


Creating file ../data/2017F/DF2017F_NormalizedHitResiduals_TIB__Layer__1.csv - File 68/133

Creating file ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TIB__layer__1.csv - File 69/133

Creating file ../data/2017F/DF2017F_NormalizedHitResiduals_TIB__Layer__2.csv - File 70/133

Creating file ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TIB__layer__2.csv - File 71/133

Creating file ../data/2017F/DF2017F_NormalizedHitResiduals_TIB__Layer__3.csv - File 72/133

Creating file ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TIB__layer__3.csv - File 73/133

Creating file ../data/2017F/DF2017F_NormalizedHitResiduals_TIB__Layer__4.csv - File 74/133

Creating file ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TIB__layer__4.csv - File 75/133

Creating file ../data/2017F/DF2017F_NormalizedHitResiduals_TID__wheel__1.csv - File 76/133

Creating file ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1.csv - File 77/133

Creating file ../